In [6]:
import csv
import math
import sys
import cv2
import numpy as np

def get_IDs(input):
    IDs_list = []
    with open(input, 'r+') as IDs:
        line = IDs.readline()
        while line:
            temp_array = line.split(',')
            if int(temp_array[0]) != 1:
                break
            else:
                IDs_list.append(temp_array[1])
            line = IDs.readline()
        IDs.close()
    return IDs_list

def create_separate_IDs(IDs_list,input):
    ID_indi_list = []
    for i in range(0,len(IDs_list)):
        ID_indi_list.clear()
        with open(input, 'r+') as IDs:
            line = IDs.readline()
            while line:
                temp_array = line.split(',')
                if IDs_list[i] == temp_array[1]:
                    ID_indi_list.append(line)
                line = IDs.readline()
            output_path="".join((output2, IDs_list[i],".csv"))
            for j in range(0,len(ID_indi_list)):
                with open(output_path, 'a') as append_to_file: 
                    original_stdout = sys.stdout
                    sys.stdout = append_to_file                                                          
                    print(ID_indi_list[j], end = '')
                    sys.stdout = original_stdout
                append_to_file.close()     
        IDs.close()

def get_xy(raw_data):
    temp_array = raw_data.split(',')
    x = float(temp_array[2])
    y = 2160-float(temp_array[3])
    return x,y 

def direction_vector(input,output):
    with open(input, 'r+') as csvfile:
        line = csvfile.readline()
        while line:
        # A point row
            p1_raw = line
        # Save current position
            pos = csvfile.tell()      
        # B point row
            for i in range(2):
                p2_raw = csvfile.readline()
            
        # A point
            p1x,p1y = get_xy(p1_raw)
            if (len(p2_raw) == 0):
                break
            else:
        # B points
                p2x,p2y = get_xy(p2_raw)   
            
            d_x,d_y = p2x-p1x,p2y-p1y
            length = math.sqrt(d_x**2+d_y**2)
            b_x=(d_x/length*pix_length)
            b_y=(d_y/length*pix_length)
            
            right_top_x = b_x+d_y/length*pix_height
            right_top_y = b_y-d_x/length*pix_height
            
            left_top_x = b_x-d_y/length*pix_height
            left_top_y = b_y+d_x/length*pix_height
            
            left_bottom_x = right_top_x * -1
            left_bottom_y = right_top_y * -1
            
            right_bottom_x = left_top_x * -1
            right_bottom_y = left_top_y * -1
            
            # Corrected to the coordinate (clockwise direction)
            right_bottom_x+=p1x
            right_bottom_y+=p1y

            left_bottom_x+=p1x
            left_bottom_y+=p1y

            left_top_x+=p1x
            left_top_y+=p1y

            right_top_x+=p1x
            right_top_y+=p1y
            
            temp_frame = p1_raw.split(',')[0]
            with open(output, 'a') as append_to_file: 
                original_stdout = sys.stdout
                sys.stdout = append_to_file                                                          
                print(temp_frame,right_bottom_x,right_bottom_y,left_bottom_x,left_bottom_y,left_top_x,left_top_y,right_top_x,right_top_y, sep=',')
                sys.stdout = original_stdout
            append_to_file.close()
            csvfile.seek(pos)
            line = csvfile.readline()
        csvfile.close()
        
        
def angle_calc(input,output):
    with open(input, 'r+') as csvfile:
        line = csvfile.readline()
        while line:
        # A point row
            p1_raw = line
        # Save current position
            pos = csvfile.tell()      
        # C point row
            for i in range(5):
                p3_raw = csvfile.readline()
        # B point row
            for i in range(5):
                p2_raw = csvfile.readline()
            
        # A point
            p1x,p1y = get_xy(p1_raw)
            if (len(p3_raw) == 0 or len(p2_raw) == 0):
                break
            else:
        # B & C points
                p2x,p2y = get_xy(p2_raw)   
                p3x,p3y = get_xy(p3_raw)
            
        # Sides (https://www.mathsisfun.com/algebra/distance-2-points.html)
            a = math.sqrt(((p2x-p3x)**2)+((p2y-p3y)**2))   # BC distance (B-C)
            b = math.sqrt(((p3x-p1x)**2)+((p3y-p1y)**2))   # CA distance (C-A)
            c = math.sqrt(((p2x-p1x)**2)+((p2y-p1y)**2))   # BA distance (B-A)
        # Law of Cosines (https://www.mathsisfun.com/algebra/trig-cosine-law.html)
            # Divide by zero
            if (-2*a*b) == 0:
                inside_angle = "ign"
                outside_angle = "ign"
            else:
                result = ((c**2)-((a**2)+(b**2)))/(-2*a*b)
                if result < 1 and result > -1:
                    inside_angle = math.degrees(math.acos(((c**2)-((a**2)+(b**2)))/(-2*a*b))) # gamma (middle point from the 3 processed row)
                    
                    outside_angle = 180 - inside_angle
            # Right angle [acos(1)]
                else:
                    inside_angle = 90
                    outside_angle = 90
            
            temp_array = p3_raw.split(',')
            with open(output, 'a') as append_to_file: 
                original_stdout = sys.stdout
                sys.stdout = append_to_file                                                          
                print(temp_array[0],temp_array[1],p3x,p3y,outside_angle, sep=',')
                sys.stdout = original_stdout
            append_to_file.close()
            csvfile.seek(pos)
            line = csvfile.readline()
        csvfile.close()
    
def valid_calc(input,output,threshold):
    with open(input, 'r+') as csvfile:
        line = csvfile.readline()
        while line:
            first = float(line.split(',')[4])
        # Save current position    
            pos = csvfile.tell()
            second = float(csvfile.readline().split(',')[4])
            third = float(csvfile.readline().split(',')[4])
            
            valid = threshold_test(first,second,threshold)
            valid+=threshold_test(first,third,threshold)

            break
            csvfile.seek(pos)
            line = csvfile.readline()
        csvfile.close()
        
def calculate_fx(x1,y1,x2,y2,angle):
    print(new_x,new_y)
    m = (y1-y2)/(x1-x2)   # Slope
    if y1 > -1:
        right_side = (m*-x1)+y1
    else:
        right_side = (m*-x1)-y1

    return m, right_side

def calc_points(x,y,angle):
  # Calculate length of each side  
    c = pix_length
    a = math.sin(math.radians(angle))*c   # y
    b = math.cos(math.radians(angle))*c   # x
    print("x: ",b,", y: ",a,"atlo: ",c)
  # Save these points as mirror point
    mid_x,mid_y = b,a 
  # Simple pythagoras  
    a = pix_length
    b = pix_height
    c = math.sqrt((a**2) + (b**2))
  # Using 'Law of Cosines' solve for angle (https://www.mathsisfun.com/algebra/trig-cosine-law.html)  
    temp_angle = (a**2+c**2-b**2)/(2*a*c)
    temp_angle = math.acos(temp_angle)
    temp_angle = math.degrees(temp_angle)
    print("temp_angle: ", temp_angle)
  # Calculate corrected angle to solve for a perpendicular point  
    temp_angle = angle - temp_angle 
    right_bottom_x = math.cos(math.radians(temp_angle))*c
    right_bottom_y = math.sin(math.radians(temp_angle))*c
    print("#################")
    print(right_bottom_x)
    print(right_bottom_y)
# Mirror across the previously calculated point (right_bottom_x,right_bottom_y)
  # Calculate for x (equation m = (x1+x2)/2)
    right_top_x = 2 * mid_x - right_bottom_x
  # Calculate for y (equation m = (y1+y2)/2)
    right_top_y = 2 * mid_y - right_bottom_y
    print("D: ", right_top_x,right_top_y)
  # Mirroring across origo
    left_top_x, left_top_y = (right_bottom_x * -1), (right_bottom_y * -1)
    
    print("E: ",left_top_x, left_top_y)
    left_bottom_x, left_bottom_y = (right_top_x * -1), (right_top_y * -1)   
    print("F: ",left_bottom_x, left_bottom_y)
    
  # Corrected to the coordinate (clockwise direction)
    right_bottom_x+=x
    right_bottom_y+=y
    
    left_bottom_x+=x
    left_bottom_y+=y
    
    left_top_x+=x
    left_top_y+=y
    
    right_top_x+=x
    right_top_y+=y
    
    return right_bottom_x, right_bottom_y,left_bottom_x,left_bottom_y,left_top_x,left_top_y,right_top_x,right_top_y
   
    
def draw_points(input):
    with open(input, 'r+') as csvfile:
        line = csvfile.readline()
        frame = float(line.split(',')[0])
        x,y = float(line.split(',')[1]),float(line.split(',')[2])
        
        x1,y1,x2,y2,x3,y3,x4,y4 = float(line.split(',')[3]),float(line.split(',')[4]),float(line.split(',')[5]),float(line.split(',')[6]),float(line.split(',')[7]),float(line.split(',')[8]),float(line.split(',')[9]),float(line.split(',')[10])
        
        image = cv2.imread('/home/wildhorse_project/test_pic.png')
        image = cv2.circle(image, (int(x),int(y)), radius=0, color=(255, 0, 0), thickness=3)
        image = cv2.circle(image, (int(x1),int(y1)), radius=0, color=(0, 0, 255), thickness=3)
        image = cv2.circle(image, (int(x2),int(y2)), radius=0, color=(0, 0, 255), thickness=3)
        image = cv2.circle(image, (int(x3),int(y3)), radius=0, color=(0, 0, 255), thickness=3)
        image = cv2.circle(image, (int(x4),int(y4)), radius=0, color=(0, 0, 255), thickness=3)

        cv2.imwrite("/home/wildhorse_project/test_pic_1.png",image)
        cv2.waitKey(0)
        cv2.destroyAllWindows
        
        csvfile.close()

def get_frame_info(input,output_dir):
    with open(input, 'r+') as coord:
        line = coord.readline()
        init_frame = line.split(',')[0]
        while line:
            if init_frame == line.split(',')[0]:
                tmp_str = "_".join((init_frame, 'frame'))
                newstr = "".join((output_dir, tmp_str))
                with open(newstr, 'a') as append_to_file: 
                    original_stdout = sys.stdout
                    sys.stdout = append_to_file                                                          
                    print(temp_array[0],temp_array[1],p3x,p3y,outside_angle, sep=' ')
                    sys.stdout = original_stdout
                append_to_file.close()
        
        coord.close()
        
def create_csv(input,actual_frame,output_dir):
    with open(input, 'r+') as coord:
        line = coord.readline()
        o_file_name = "".join(("_".join(("V180824_2_12fps_4k",str(actual_frame).zfill(4))),".txt"))
        while line:
            if line.split(',')[0] == str(actual_frame):
                new_str = "".join((output_dir,o_file_name))
                x1,y1 = int(float(line.split(',')[1])),int(float(line.split(',')[2]))
                x2,y2 = int(float(line.split(',')[3])),int(float(line.split(',')[4]))
                x3,y3 = int(float(line.split(',')[5])),int(float(line.split(',')[6]))
                x4,y4 = int(float(line.split(',')[7])),int(float(line.split(',')[8]))
                with open(new_str, 'a') as append_to_file: 
                    original_stdout = sys.stdout
                    sys.stdout = append_to_file                                                          
                    print(x1,y1,x2,y2,x3,y3,x4,y4,"horse","0", sep=' ')
                    sys.stdout = original_stdout
                append_to_file.close()
                break
            line = coord.readline()
        coord.close()
# Necessary parameters    
input = '/home/wildhorse_project/horse_coordinates/horse_coordinates/180824/horse_coordinatesID.txt'
output2 = '/home/wildhorse_project/horse_coordinates/horse_coordinates/180824/separate_IDs/'
max_frame = 4149
threshold = 30
pix_length = 32 # Length of horse
pix_height = 16 # Height of horse

pix_height/=2
pix_length/=2

#input_angle_path='/home/wildhorse_project/horse_coordinates/horse_coordinates/180824/separate_IDs/H165.csv'
#output_angle_path='/home/wildhorse_project/horse_coordinates/horse_coordinates/180824/teszt_angle.txt'
#angle_calc(input_angle_path,output_angle_path)

print("Get frame IDs...", sep="")

input_coord='/home/wildhorse_project/horse_coordinates/horse_coordinates/180824/horse_coordinates.txt'
output_dir='/home/wildhorse_project/horse_coordinates/horse_coordinates/180824/separate_frames/'
#get_frame_info(input_coord, output_dir)

print("Creating list of IDs...", sep="")
IDs_list = get_IDs(input)
print("    DONE")

#Step3
print("Create coordinates...",sep="")
for i in range(1,max_frame):
    for j in range(0,len(IDs_list)):
        input_ID="".join((output2,IDs_list[j],"_coordinates_2.csv"))
        create_csv(input_ID,i,output_dir)
    print(i, "DONE",sep=" ")
print("    DONE")
    


#Step3
#print("Calculate direction vectors...",sep="")
#for i in range(0,len(IDs_list)):
#    input_angle_path="".join((output2,IDs_list[i],".csv"))
#    output_angle_path="".join((output2,IDs_list[i],"_coordinates_2.csv"))
#    direction_vector(input_angle_path,output_angle_path)
#print("    DONE")




        
    
    

Get frame IDs...
Creating list of IDs...
    DONE
Create coordinates...
1 DONE
2 DONE
3 DONE
4 DONE
5 DONE
6 DONE
7 DONE
8 DONE
9 DONE
10 DONE
11 DONE
12 DONE
13 DONE
14 DONE
15 DONE
16 DONE
17 DONE
18 DONE
19 DONE
20 DONE
21 DONE
22 DONE
23 DONE
24 DONE
25 DONE
26 DONE
27 DONE
28 DONE
29 DONE
30 DONE
31 DONE
32 DONE
33 DONE
34 DONE
35 DONE
36 DONE
37 DONE
38 DONE
39 DONE
40 DONE
41 DONE
42 DONE
43 DONE
44 DONE
45 DONE
46 DONE
47 DONE
48 DONE
49 DONE
50 DONE
51 DONE
52 DONE
53 DONE
54 DONE
55 DONE
56 DONE
57 DONE
58 DONE
59 DONE
60 DONE
61 DONE
62 DONE
63 DONE
64 DONE
65 DONE
66 DONE
67 DONE
68 DONE
69 DONE
70 DONE
71 DONE
72 DONE
73 DONE
74 DONE
75 DONE
76 DONE
77 DONE
78 DONE
79 DONE
80 DONE
81 DONE
82 DONE
83 DONE
84 DONE
85 DONE
86 DONE
87 DONE
88 DONE
89 DONE
90 DONE
91 DONE
92 DONE
93 DONE
94 DONE
95 DONE
96 DONE
97 DONE
98 DONE
99 DONE
100 DONE
101 DONE
102 DONE
103 DONE
104 DONE
105 DONE
106 DONE
107 DONE
108 DONE
109 DONE
110 DONE
111 DONE
112 DONE
113 DONE
114 DONE
115 DONE
1

916 DONE
917 DONE
918 DONE
919 DONE
920 DONE
921 DONE
922 DONE
923 DONE
924 DONE
925 DONE
926 DONE
927 DONE
928 DONE
929 DONE
930 DONE
931 DONE
932 DONE
933 DONE
934 DONE
935 DONE
936 DONE
937 DONE
938 DONE
939 DONE
940 DONE
941 DONE
942 DONE
943 DONE
944 DONE
945 DONE
946 DONE
947 DONE
948 DONE
949 DONE
950 DONE
951 DONE
952 DONE
953 DONE
954 DONE
955 DONE
956 DONE
957 DONE
958 DONE
959 DONE
960 DONE
961 DONE
962 DONE
963 DONE
964 DONE
965 DONE
966 DONE
967 DONE
968 DONE
969 DONE
970 DONE
971 DONE
972 DONE
973 DONE
974 DONE
975 DONE
976 DONE
977 DONE
978 DONE
979 DONE
980 DONE
981 DONE
982 DONE
983 DONE
984 DONE
985 DONE
986 DONE
987 DONE
988 DONE
989 DONE
990 DONE
991 DONE
992 DONE
993 DONE
994 DONE
995 DONE
996 DONE
997 DONE
998 DONE
999 DONE
1000 DONE
1001 DONE
1002 DONE
1003 DONE
1004 DONE
1005 DONE
1006 DONE
1007 DONE
1008 DONE
1009 DONE
1010 DONE
1011 DONE
1012 DONE
1013 DONE
1014 DONE
1015 DONE
1016 DONE
1017 DONE
1018 DONE
1019 DONE
1020 DONE
1021 DONE
1022 DONE
1023 DONE
1024

1744 DONE
1745 DONE
1746 DONE
1747 DONE
1748 DONE
1749 DONE
1750 DONE
1751 DONE
1752 DONE
1753 DONE
1754 DONE
1755 DONE
1756 DONE
1757 DONE
1758 DONE
1759 DONE
1760 DONE
1761 DONE
1762 DONE
1763 DONE
1764 DONE
1765 DONE
1766 DONE
1767 DONE
1768 DONE
1769 DONE
1770 DONE
1771 DONE
1772 DONE
1773 DONE
1774 DONE
1775 DONE
1776 DONE
1777 DONE
1778 DONE
1779 DONE
1780 DONE
1781 DONE
1782 DONE
1783 DONE
1784 DONE
1785 DONE
1786 DONE
1787 DONE
1788 DONE
1789 DONE
1790 DONE
1791 DONE
1792 DONE
1793 DONE
1794 DONE
1795 DONE
1796 DONE
1797 DONE
1798 DONE
1799 DONE
1800 DONE
1801 DONE
1802 DONE
1803 DONE
1804 DONE
1805 DONE
1806 DONE
1807 DONE
1808 DONE
1809 DONE
1810 DONE
1811 DONE
1812 DONE
1813 DONE
1814 DONE
1815 DONE
1816 DONE
1817 DONE
1818 DONE
1819 DONE
1820 DONE
1821 DONE
1822 DONE
1823 DONE
1824 DONE
1825 DONE
1826 DONE
1827 DONE
1828 DONE
1829 DONE
1830 DONE
1831 DONE
1832 DONE
1833 DONE
1834 DONE
1835 DONE
1836 DONE
1837 DONE
1838 DONE
1839 DONE
1840 DONE
1841 DONE
1842 DONE
1843 DONE


2564 DONE
2565 DONE
2566 DONE
2567 DONE
2568 DONE
2569 DONE
2570 DONE
2571 DONE
2572 DONE
2573 DONE
2574 DONE
2575 DONE
2576 DONE
2577 DONE
2578 DONE
2579 DONE
2580 DONE
2581 DONE
2582 DONE
2583 DONE
2584 DONE
2585 DONE
2586 DONE
2587 DONE
2588 DONE
2589 DONE
2590 DONE
2591 DONE
2592 DONE
2593 DONE
2594 DONE
2595 DONE
2596 DONE
2597 DONE
2598 DONE
2599 DONE
2600 DONE
2601 DONE
2602 DONE
2603 DONE
2604 DONE
2605 DONE
2606 DONE
2607 DONE
2608 DONE
2609 DONE
2610 DONE
2611 DONE
2612 DONE
2613 DONE
2614 DONE
2615 DONE
2616 DONE
2617 DONE
2618 DONE
2619 DONE
2620 DONE
2621 DONE
2622 DONE
2623 DONE
2624 DONE
2625 DONE
2626 DONE
2627 DONE
2628 DONE
2629 DONE
2630 DONE
2631 DONE
2632 DONE
2633 DONE
2634 DONE
2635 DONE
2636 DONE
2637 DONE
2638 DONE
2639 DONE
2640 DONE
2641 DONE
2642 DONE
2643 DONE
2644 DONE
2645 DONE
2646 DONE
2647 DONE
2648 DONE
2649 DONE
2650 DONE
2651 DONE
2652 DONE
2653 DONE
2654 DONE
2655 DONE
2656 DONE
2657 DONE
2658 DONE
2659 DONE
2660 DONE
2661 DONE
2662 DONE
2663 DONE


3384 DONE
3385 DONE
3386 DONE
3387 DONE
3388 DONE
3389 DONE
3390 DONE
3391 DONE
3392 DONE
3393 DONE
3394 DONE
3395 DONE
3396 DONE
3397 DONE
3398 DONE
3399 DONE
3400 DONE
3401 DONE
3402 DONE
3403 DONE
3404 DONE
3405 DONE
3406 DONE
3407 DONE
3408 DONE
3409 DONE
3410 DONE
3411 DONE
3412 DONE
3413 DONE
3414 DONE
3415 DONE
3416 DONE
3417 DONE
3418 DONE
3419 DONE
3420 DONE
3421 DONE
3422 DONE
3423 DONE
3424 DONE
3425 DONE
3426 DONE
3427 DONE
3428 DONE
3429 DONE
3430 DONE
3431 DONE
3432 DONE
3433 DONE
3434 DONE
3435 DONE
3436 DONE
3437 DONE
3438 DONE
3439 DONE
3440 DONE
3441 DONE
3442 DONE
3443 DONE
3444 DONE
3445 DONE
3446 DONE
3447 DONE
3448 DONE
3449 DONE
3450 DONE
3451 DONE
3452 DONE
3453 DONE
3454 DONE
3455 DONE
3456 DONE
3457 DONE
3458 DONE
3459 DONE
3460 DONE
3461 DONE
3462 DONE
3463 DONE
3464 DONE
3465 DONE
3466 DONE
3467 DONE
3468 DONE
3469 DONE
3470 DONE
3471 DONE
3472 DONE
3473 DONE
3474 DONE
3475 DONE
3476 DONE
3477 DONE
3478 DONE
3479 DONE
3480 DONE
3481 DONE
3482 DONE
3483 DONE
